The goal of the data cleaning is to have final file that can be used for exploraty data analysis.
Steps:
 - take care of duplicated rows and empty values
 - convert event_type == 'purchase' into a feature for the last view before the purchase
 - convert event_type == 'cart' into two features before and after a view.

# Libraries

In [1]:
import pandas as pd
import os
import sys

sys.path.append(os.path.abspath(".."))
from utils import helper_functions as hf

# Uploading the data

In [2]:
filepath = '../data/events.csv' 
df = pd.read_csv(filepath) 
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-09-24 11:57:06 UTC,view,1996170,2144415922528452715,electronics.telephone,NaN,31.90,1515915625519388267,LJuJVLEjPT
1,2020-09-24 11:57:26 UTC,view,139905,2144415926932472027,computers.components.cooler,zalman,17.16,1515915625519380411,tdicluNnRY
2,2020-09-24 11:57:27 UTC,view,215454,2144415927158964449,NaN,NaN,9.81,1515915625513238515,4TMArHtXQy
3,2020-09-24 11:57:33 UTC,view,635807,2144415923107266682,computers.peripherals.printer,pantum,113.81,1515915625519014356,aGFYrNgC08
4,2020-09-24 11:57:36 UTC,view,3658723,2144415921169498184,NaN,cameronsino,15.87,1515915625510743344,aa4mmk0kwQ


In [3]:
df.info() # columns, non-null counts, dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 885129 entries, 0 to 885128
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   event_time     885129 non-null  object 
 1   event_type     885129 non-null  object 
 2   product_id     885129 non-null  int64  
 3   category_id    885129 non-null  int64  
 4   category_code  648910 non-null  object 
 5   brand          672765 non-null  object 
 6   price          885129 non-null  float64
 7   user_id        885129 non-null  int64  
 8   user_session   884964 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 60.8+ MB


In [4]:
df.shape #rows and columns

(885129, 9)

In [5]:
hf.check_data(df) #custom function to check for duplicate, nulls and empty values

There are 655 duplicate rows, 448748 empty values and 0 empty spaces

Duplicate Rows:
0.07% of duplicated rows. View some of them below:
                     event_time event_type  product_id          category_id  \
409189  2020-12-07 19:55:11 UTC       view     3948734  2144415926966026460   
847405  2021-02-22 04:20:22 UTC       view     1674454  2144415922528452715   
670861  2021-01-24 11:07:04 UTC       view     4183836  2144415923535085701   

                   category_code    brand   price              user_id  \
409189  computers.components.cpu      amd  631.24  1515915625553959968   
847405     electronics.telephone      NaN   11.86  1515915625608290672   
670861                       NaN  samsung   75.76  1515915625596535958   

       user_session  
409189   hM21oRwsVd  
847405   DjzzEhn7pt  
670861   QwJgFowxYe  

NaN Rows:
26.69% of NaNs in the category_code column
23.99% of NaNs in the brand column
0.02% of NaNs in the user_session column


In [6]:
# checking unique values per column
df.nunique()

event_time       845041
event_type            3
product_id        53453
category_id         718
category_code       107
brand               999
price             12422
user_id          407283
user_session     490398
dtype: int64

In [7]:
# checking the three unique event types of this dataframe
df.event_type.value_counts()

event_type
view        793748
cart         54035
purchase     37346
Name: count, dtype: int64

### Converting the event_time to a timestamp

In [8]:
# convert the 'event_time' column to datetime format
df['event_time'] = pd.to_datetime(df['event_time'])

### Checking the duplications
ALthough only 0.07% of the rows are duplicated, we want to take a closer look at them.

In [9]:
perc = str(round(df.duplicated().sum() / len(df) * 100, 2))
print(f"{perc}% of duplicated rows.")

0.07% of duplicated rows.


In [10]:
# adding all duplicated rows including the original row to a new DataFrame
df_duplicates = df[df.duplicated(keep=False)]
# adding those duplicated rows to a new CSV file
df_duplicates.to_csv('../data/cleaning/duplicates.csv', index=False)
# checking for which event type this duplication is happening
df_duplicates['event_type'].value_counts()

event_type
view        1272
cart           6
purchase       6
Name: count, dtype: int64

In [11]:
# checking for which time frame the duplication is happening
print(df_duplicates['event_time'].min()), print(df_duplicates['event_time'].max())
# checking for which user id this duplication is happening
print(df_duplicates['user_id'].value_counts().head(10))


2020-09-24 13:51:07+00:00
2021-02-28 14:18:02+00:00
user_id
1515915625530338453    14
1515915625598503848    13
1515915625551553663    12
1515915625561515907    12
1515915625512176731     8
1515915625595650588     8
1515915625596017017     7
1515915625548733800     6
1515915625540086964     6
1515915625545471734     6
Name: count, dtype: int64


#### Assessment of duplicated rows:
As it only happens < 1% of the times it seems not to be a normal process but rather a glitch in the system. We would need to ask the developers what it means to get the same event twice. If exactly the same event with all same columns has been recorded twice although it just happened once, then we can safely remove the duplication as there is no information lost.
We are going with this assumption and thus removing the duplicated data.

In [12]:
# drop duplicated rows
df = df.drop_duplicates()
# check if duplicates are removed
df.duplicated().sum()

0

### Checking the NAN values

In [13]:
# printing the number of NaN values per column
nan_values = df.isna().sum()
print("NaN Rows:")
for x in nan_values[nan_values > 0].index:
    perc = str(round(df[x].isna().sum() / len(df) * 100, 2))
    print(f"{perc}% of NaNs in the {x} column")

NaN Rows:
26.69% of NaNs in the category_code column
24.0% of NaNs in the brand column
0.02% of NaNs in the user_session column


In [14]:
# identify rows with any NaN values and write to a new dataframe
df_nan = df[df.isna().any(axis=1)].reset_index(drop=True)
df_nan.to_csv('../data/cleaning/nan_values_all.csv', index=False)
df_nan.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-09-24 11:57:06+00:00,view,1996170,2144415922528452715,electronics.telephone,NaN,31.90,1515915625519388267,LJuJVLEjPT
1,2020-09-24 11:57:27+00:00,view,215454,2144415927158964449,NaN,NaN,9.81,1515915625513238515,4TMArHtXQy
2,2020-09-24 11:57:36+00:00,view,3658723,2144415921169498184,NaN,cameronsino,15.87,1515915625510743344,aa4mmk0kwQ
3,2020-09-24 11:58:23+00:00,view,3791349,2144415935086199225,computers.desktop,NaN,215.41,1515915625519388877,J1t6sIYXiV
4,2020-09-24 11:58:25+00:00,view,657859,2144415939431498289,NaN,NaN,34.17,1515915625519320570,HEl15U7JVy


#### Checking empty brand and category_code

Some product_ids have no brand or category code. The question is whether somewhere else in the dataframe the same product_id does have a brand name or category_code. For this reason, I create a unique dictionary with all product_ids and what brand or category code they map too.

In [15]:
# create a lookup dictionary for product_id to (brand, category_code)
product_dict = {}

# iterate over the rows of the DataFrame
for _, row in df_nan[['product_id', 'brand', 'category_code']].drop_duplicates().iterrows():
    product_id = row['product_id']
    brand = row['brand']
    category_code = row['category_code']
    
    # fill values if possible
    if pd.isna(brand):
        brand = df.loc[df['product_id'] == product_id, 'brand'].dropna().unique()
        brand = brand[0] if len(brand) > 0 else None
    if pd.isna(category_code):
        category_code = df.loc[df['product_id'] == product_id, 'category_code'].dropna().unique()
        category_code = category_code[0] if len(category_code) > 0 else None
    
    # add to dictionary
    product_dict[product_id] = {"brand" : brand , "category_code" : category_code}

Then I compare the dictionary with my df_nan dataframe (any row that has an empty value) to see if anything could be mapped.

In [16]:
unique_product_ids_with_none_brand_fromDict = len([product_id for product_id, details in product_dict.items() if details['brand'] is None])
unique_product_ids_with_none_category_code_fromDict = len([product_id for product_id, details in product_dict.items() if details['category_code'] is None])
unique_product_ids_with_none_brand_fromDFNAN = df_nan[df_nan['brand'].isna()].product_id.nunique()
unique_product_ids_with_none_category_code_fromDFNAN = df_nan[df_nan['category_code'].isna()].product_id.nunique()

print(
    f"Is remapping needed for 'brand': {'Yes' if unique_product_ids_with_none_brand_fromDict != unique_product_ids_with_none_brand_fromDFNAN else 'No'}, "
    f"Is remapping needed for 'category_code': {'Yes' if unique_product_ids_with_none_category_code_fromDict != unique_product_ids_with_none_category_code_fromDFNAN else 'No'}"
)

Is remapping needed for 'brand': No, Is remapping needed for 'category_code': No


As no mapping can be found, we can fill all empty values with "unknown"

In [17]:
df['brand'] = df['brand'].fillna('Unknown')
df['category_code'] = df['category_code'].fillna('Unknown')

In [18]:
nan_values = df.isna().sum()
print("NaN Rows:")
for x in nan_values[nan_values > 0].index:
    perc = str(round(df[x].isna().sum() / len(df) * 100, 2))
    print(f"{perc}% of NaNs in the {x} column")

NaN Rows:
0.02% of NaNs in the user_session column


#### Checking the empty user sessions

Taking a look at user session for one user to discover the "flow".

In [19]:
pd.set_option('display.max_row', None) # display all rows
user_data = df[df['user_id'] == 1515915625556087775]
user_data.sort_values(by='event_time').head(20)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
414574,2020-12-09 01:47:19+00:00,view,743179,2144415935631458761,construction.tools.soldering,kada,50.32,1515915625556087775,DzzkWOAW9g
414597,2020-12-09 02:22:53+00:00,view,743179,2144415935631458761,construction.tools.soldering,kada,50.32,1515915625556087775,BejOXRngEW
414598,2020-12-09 02:23:21+00:00,cart,743179,2144415935631458761,construction.tools.soldering,kada,50.32,1515915625556087775,BejOXRngEW
414599,2020-12-09 02:24:09+00:00,view,743179,2144415935631458761,construction.tools.soldering,kada,50.32,1515915625556087775,BejOXRngEW
414600,2020-12-09 02:24:46+00:00,view,743182,2144415935631458761,construction.tools.soldering,kada,65.08,1515915625556087775,BejOXRngEW
414613,2020-12-09 02:34:08+00:00,cart,743182,2144415935631458761,construction.tools.soldering,kada,65.08,1515915625556087775,BejOXRngEW
414617,2020-12-09 02:35:32+00:00,purchase,743182,2144415935631458761,construction.tools.soldering,kada,65.08,1515915625556087775,BejOXRngEW
414623,2020-12-09 02:39:52+00:00,view,743182,2144415935631458761,construction.tools.soldering,kada,65.08,1515915625556087775,BejOXRngEW
415218,2020-12-09 06:35:39+00:00,view,743179,2144415935631458761,construction.tools.soldering,kada,50.32,1515915625556087775,BejOXRngEW
415293,2020-12-09 06:56:17+00:00,view,564686,2144415935631458761,construction.tools.soldering,Unknown,52.38,1515915625556087775,BejOXRngEW


It is unclear what user_session to me what user session means as it jumps from one session to the other and back and forth. Without any further insights onto how user_session is created it is not useful. Will drop that column and focus on user_id and product_id sequentially in time.

In [20]:
df = df.drop('user_session', axis=1) # drop the user_session column

## Sorting the dataframe sequentially by user_id and product_id

In [21]:
df = df.sort_values(by=['user_id', 'product_id', 'event_time'])

# Preparting Data for the last view before purchase analysis

#### Creating a sanity check example dataset to validate the methods are working

In [22]:
df[df['event_type'] == 'purchase'].user_id.value_counts().head(10) # grabbing a few users that had a purchase event

user_id
1515915625601579158    56
1515915625591659523    49
1515915625596534847    45
1515915625604175669    43
1515915625605054644    42
1515915625599634258    39
1515915625521745364    36
1515915625603840072    33
1515915625603113992    28
1515915625540197343    24
Name: count, dtype: int64

In [23]:
# putting the users into a list for the sanity check
user_ids_to_test = (1515915625601579158, 1515915625591659523, 1515915625596534847, 1515915625604175669, 1515915625605054644)

# creating a Test DataFrame with only those users from the list and saving to a csv file
test_df = df[df['user_id'].isin(user_ids_to_test)]
test_df = test_df.sort_values(by=['user_id', 'product_id', 'event_time'])
test_df.to_csv('sanity_check/test_df.csv', index=False)

# The code below performed on the whole dataframe has been sanity checked in notebooks/sanity_check/example.ipynb. 
# Feel free to check there the logic which is then applied to the whole dataframe below.

## Converting the purchase event row into a feature of the previous view event
We want every view before the purchase to be marked if it was the last view before purchase per user_id and product_id sequentially (in case the same user_id and product_id combinations buys the same product_id on multiple occassions)

In [24]:
# at the beginning the new columns is 0 for everyone
df['last_view_before_purchase'] = 0

# function to mark the last view before a purchase event
def mark_last_view_before_purchase(group):
    purchase_indices = group.index[group['event_type'] == 'purchase'].tolist()
    if not purchase_indices:
        return group

    for purchase_index in purchase_indices:
        # find the 'last view' before 'purchase'
        views_before_purchase = group[(group.index < purchase_index) & (group['event_type'] == 'view')]
        if not views_before_purchase.empty:
            last_view_index = views_before_purchase.index[-1]
            group.at[last_view_index, 'last_view_before_purchase'] = 1

    return group

df = df.groupby(['user_id', 'product_id']).apply(mark_last_view_before_purchase)

/var/folders/pm/ddtx28cd2hn37jlq10mrtz700000gn/T/ipykernel_60012/2137655026.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['user_id', 'product_id']).apply(mark_last_view_before_purchase)


In [25]:
print("Current index levels:", test_df.index.names)

Current index levels: [None]


In [26]:
df = df.reset_index(drop=True)

## Converting the cart event row into a feature of the previous view event
We want every view before the cart to be marked if it was the last view before cart per user_id and product_id sequentially 

In [27]:
# at the beginning the new column is 0 for everyone
df['last_view_before_cart'] = 0

# function to mark the last view before a cart event
def mark_last_view_before_cart(group):
    cart_indices = group.index[group['event_type'] == 'cart'].tolist()
    if not cart_indices:
        return group

    for cart_index in cart_indices:
        # find the 'last view' before 'cart'
        views_before_cart = group[(group.index < cart_index) & (group['event_type'] == 'view')]
        if not views_before_cart.empty:
            last_view_index = views_before_cart.index[-1]
            group.at[last_view_index, 'last_view_before_cart'] = 1

    return group

# apply the function to each group of user_id and product_id
df = df.groupby(['user_id', 'product_id']).apply(mark_last_view_before_cart)

/var/folders/pm/ddtx28cd2hn37jlq10mrtz700000gn/T/ipykernel_60012/784477939.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['user_id', 'product_id']).apply(mark_last_view_before_cart)


In [28]:
df = df.reset_index(drop=True)

In [29]:
#at the beginning the new column is 0 for everyone
df['already_in_cart'] = 0

# function to mark views that follow a cart event without a purchase in between
def mark_view_after_cart(group):
    # get indices of cart and view events
    cart_indices = group.index[group['event_type'] == 'cart'].tolist()
    view_indices = group.index[group['event_type'] == 'view'].tolist()
    
    # iterate over each cart event index
    for cart_index in cart_indices:
        # find the first view after the cart event
        for view_index in view_indices:
            if view_index > cart_index:
                # check if there is a purchase between the cart and the view
                purchase_between = group.loc[cart_index + 1:view_index - 1]['event_type'].eq('purchase').any()
                # if there is no purchase event in between, mark the view
                if not purchase_between:
                    group.at[view_index, 'already_in_cart'] = 1
                break  # only mark the first view after the cart

    return group

# apply the function to each group of user_id and product_id
df = df.groupby(['user_id', 'product_id']).apply(mark_view_after_cart)

/var/folders/pm/ddtx28cd2hn37jlq10mrtz700000gn/T/ipykernel_60012/896061175.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['user_id', 'product_id']).apply(mark_view_after_cart)


Save final df as csv for further exploratory data analysis

In [30]:
df.to_csv('../data/events_cleaned.csv', index=False)

In [31]:
df.head()

event_time event_type  \
user_id             product_id                                          
1515915625353226922 4101974    0 2020-10-29 11:28:35+00:00       view   
1515915625353230067 3506650    1 2020-10-06 06:30:32+00:00       view   
1515915625353230683 124883     2 2020-11-09 08:52:51+00:00       view   
                    125325     3 2020-11-09 09:08:53+00:00       view   
                    254763     4 2020-11-18 10:51:35+00:00       view   

                                  product_id          category_id  \
user_id             product_id                                      
1515915625353226922 4101974    0     4101974  2144415939364389423   
1515915625353230067 3506650    1     3506650  2144415935673401802   
1515915625353230683 124883     2      124883  2144415924424278172   
                    125325     3      125325  2144415924424278172   
                    254763     4      254763  2144415924424278172   

                                               category_code     brand  price  \
user_id             product_id                                                  
1515915625353226922 4101974    0          electronics.clocks     honor  76.48   
1515915625353230067 3506650    1                     Unknown    kester  28.98   
1515915625353230683 124883     2  electronics.audio.acoustic  logitech  23.90   
                    125325     3  electronics.audio.acoustic  logitech  23.90   
                    254763     4  electronics.audio.acoustic  creative  74.24   

                                              user_id  \
user_id             product_id                          
1515915625353226922 4101974    0  1515915625353226922   
1515915625353230067 3506650    1  1515915625353230067   
1515915625353230683 124883     2  1515915625353230683   
                    125325     3  1515915625353230683   
                    254763     4  1515915625353230683   

                                  last_view_before_purchase  \
user_id             product_id                                
1515915625353226922 4101974    0                          0   
1515915625353230067 3506650    1                          0   
1515915625353230683 124883     2                          0   
                    125325     3                          0   
                    254763     4                          0   

                                  last_view_before_cart  already_in_cart  
user_id             product_id                                            
1515915625353226922 4101974    0                      0                0  
1515915625353230067 3506650    1                      0                0  
1515915625353230683 124883     2                      0                0  
                    125325     3                      0                0  
                    254763     4                      0                0